In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.cm import get_cmap
from matplotlib.colors import BoundaryNorm
import numpy as np
%matplotlib widget

# 1. Plane wave in free field

We play around with the plane wave solution $p \,\propto\, \mathrm{e}^{\mathrm{j}(-k_x x - k_y y + \omega t)}$ of the linear, homogeneous wave equation

$$\mathrm{div}\,\mathrm{grad}\,p(\mathbf{x},t) - \frac{1}{c^2} \frac{\partial^2 p(\mathbf{x},t)}{\partial t^2} = 0$$

in the $xy$-plane denoting speed of sound with $c$.

Here we assume no radiation into $z$-direction, i.e. $k_z=0$.
Hence, the dispersion relation reduces to

$$k_x^2+k_y^2 = \left(\frac{\omega}{c}\right)^2.$$

In vector notation, using

\begin{equation}
\mathbf{k} = 
\begin{pmatrix}
k_x\\k_y\\k_z=0
\end{pmatrix},
\qquad
\mathbf{x} = 
\begin{pmatrix}
x\\y\\z=0
\end{pmatrix},
\end{equation}

we can write the considered plane wave solution in dot product fashion as

$$p \,\propto\, \mathrm{e}^{\mathrm{j}(-\mathbf{k}^\mathrm{T} \mathbf{x} + \omega t)},$$

where $\mathbf{k}$ indicates the **traveling/propagating direction** of the plane wave when choosing above $\mathrm{e}^{+\mathrm{j} \omega t}$ time convention.

We figure, that $k_x^2+k_y^2 = \left(\frac{\omega}{c}\right)^2$ is like Pythagorean theorem for right triangle.
So, we can choose two of the three squared quantities, the third one is then fully determined.
If the (angular) frequency $\omega$ was already chosen (which we usually start with), we are left to choose only $k_x$ or $k_y$.
This finally leaves us with the choice of radiation angle $\phi$, which is encoded in the equations

$$k_x = \frac{\omega}{c}\cos(\phi)\qquad k_y = \frac{\omega}{c}\sin(\phi)$$

for propagating plane waves (we do not deal here with so called evanescent waves, i.e. we assume $k_x\in\mathbb{R}$ and $k_y\in\mathbb{R}$).

Let's check how this can be implemented, plotted and animated to gain some insight into simple wave phenomena.

In [ ]:
c = 343  # m/s, speed of sound
wave_length = 1  # m
pA = 1  # N / m^2, arbitrarily chosen pressure amplitude
radiation_angle = 0  # deg, propagating direction within xy-plane

In [ ]:
# square xy-plane
N = 2**6
xylim = 2
x, y = np.meshgrid(np.linspace(-xylim, xylim, N, endpoint=True),
                   np.linspace(-xylim, xylim, N, endpoint=True),
                   indexing='ij')

In [ ]:
w_c = 2*np.pi / wave_length  # rad/m
kx = w_c * np.cos(np.deg2rad(radiation_angle))
ky = w_c * np.sin(np.deg2rad(radiation_angle))
print('dispersion relation ok?', np.allclose(kx**2 + ky**2, w_c**2))
w = w_c*c  # rad/s, angular frequency

In [ ]:
# plane wave in xy-dimension using exp(+j w t) time convention
p = pA * np.exp(-1j * (kx*x + ky*y))

## 1.1 Sound field snapshot using plot_surface

In [ ]:
col_tick = np.linspace(-pA, +pA, 255, endpoint=True)
cmap_lin = get_cmap('bwr').copy()
norm_lin = BoundaryNorm(col_tick, cmap_lin.N)

fig, ax = plt.subplots(figsize=(5, 4),
                       subplot_kw={"projection": "3d"})
surf = ax.plot_surface(x, y, np.real(p),
                       cmap=cmap_lin,
                       norm=norm_lin,
                       antialiased=True)
ax.set_xlabel('x / m')
ax.set_ylabel('y / m')
ax.azim = -90
ax.elev = 90
fig.colorbar(surf,
             ticks=np.linspace(-pA, +pA, 5),
             shrink=0.6, aspect=15)

## 1.2 Sound field snapshot using pcolormesh

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), nrows=1, ncols=1)
surf = ax.pcolormesh(x, y, np.real(p),
                     cmap=cmap_lin,
                     norm=norm_lin,
                     antialiased=True)
ax.set_xlabel('x / m')
ax.set_ylabel('y / m')
ax.set_xticks(np.linspace(-2, 2, 5))
ax.set_yticks(np.linspace(-2, 2, 5))
ax.grid(True)
fig.colorbar(surf, ticks=np.linspace(-pA, +pA, 5),
             shrink=0.9, aspect=20)

## 1.3 Animation of sound field using plot_surface

In [ ]:
n_periods = 2
n_samples_per_period = 16
Ts = 2*np.pi / (w*n_samples_per_period)  # s, sampling period


def init():
    ax.set_xlabel('x / m')
    ax.set_ylabel('y / m')
    ax.azim = -90
    ax.elev = 90


def update(frame):
    ax.clear()
    init()
    ax.plot_surface(x, y, np.real(p*np.exp(+1j*w*Ts*frame)),
                    cmap=cmap_lin,
                    norm=norm_lin,
                    antialiased=True)


fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
update(0)
init()

ani = FuncAnimation(fig, update,
                    frames=np.arange(n_periods*n_samples_per_period+1),
                    blit=True, interval=50,
                    repeat=False)
plt.show()

## 1.4 Animation of sound field using pcolormesh

In [ ]:
n_periods = 2
n_samples_per_period = 16
Ts = 2*np.pi / (w*n_samples_per_period)  # s, sampling period


def init():
    ax.set_xlabel('x / m')
    ax.set_ylabel('y / m')
    ax.set_xticks(np.linspace(-2, 2, 5))
    ax.set_yticks(np.linspace(-2, 2, 5))


def update(frame):
    ax.clear()
    init()
    ax.pcolormesh(x, y, np.real(p*np.exp(+1j*w*Ts*frame)),
                  cmap=cmap_lin,
                  norm=norm_lin,
                  antialiased=True)
    ax.set_title('elapsed time {time:4.3f} ms'.format(time=Ts*frame*1000))


fig, ax = plt.subplots(figsize=(4, 4), nrows=1, ncols=1)
update(0)
init()

ani = FuncAnimation(fig, update,
                    frames=np.arange(n_periods*n_samples_per_period+1),
                    blit=True, interval=50,
                    repeat=False)
plt.show()

# 2. Exercise

Another solution to the linear wave equation

$$\mathrm{div}\,\mathrm{grad} p(\mathbf{x},t) - \frac{1}{c^2} \frac{\partial^2 p(\mathbf{x},t)}{\partial t^2} = 0$$

except for the point $\mathbf{x} = (0,0,0)^\mathrm{T}$ is

$$p \,\propto\, \frac{1}{4 \pi} \, \frac{\mathrm{e}^{\mathrm{j}(-\frac{\omega}{c} r+\omega t)}}{r}$$

with $r = \sqrt{x^2+y^2+z^2}$.

## 2.1 Tasks

1. Adapt the above source code for this wave equation solution.
2. Plot the sound field.
3. Animate the sound field.
4. What kind of wave is this? We might also want to check the $xz$ and $yz$ plane to figure this out.
5. What happens if we reverse the sign of the time phasor for the animation? Explain the observed phenomenon.

### 2.1.1 Sound field snapshot using pcolormesh

In [ ]:
# r = 
# p = 

### 2.1.2 Sound field snapshot using pcolormesh

In [ ]:
#...

### 2.1.3 Animation of sound field using pcolormesh

In [ ]:
#...

### 2.1.4

In [ ]:
#...

### 2.1.5

In [ ]:
#...

# 3. Plane wave with certain boundary conditions lead to a standing wave

We consider 1D radiation along $x$-axis. Hence, $k_x^2 = \left(\frac{\omega}{c}\right)^2$, $k_y=$, $k_z=0$.

At $-l$ (with $l>0$), i.e. at some point on the negative $x$-axis, there is an ideal membrane located, which injects the harmonic velocity $v_A$ into a (long, thin) tube.
This tube physically enforces our aimed 1D propagation of the wave, we assume no energy loss. 
For this, the height and width of this tube must be much, much smaller than the wavelength under consideration.
The length of the tube along the $x$-axis should exhibit at least half of the wavelength, but could be longer.

At $x=0$ there is a sound rigid wall (the choice of $0$ makes calculus much simpler).
This results in our considered boundary condition $v(x=0) = 0$.
Sound rigid walls force velocity to zero at the wall, walls are rather not moving by impinging sound waves, right?!


In Ch. 2 of https://link.springer.com/book/10.1007/978-3-662-47704-5 we find the derivation of the standing wave phenomena which we are dealing here.

We need the density of air $\rho_0$ in the below equations.

## 3.1 Solution standing wave

The solution is

$$p(x,t) = 2 p_A \cos(k_x x) \mathrm{e}^{\mathrm{j} \omega t}$$
$$v(x,t) = 2 p_A \frac{-\mathrm{j}}{\rho_0 c}\sin(k_x x) \mathrm{e}^{\mathrm{j} \omega t},$$

or written with our used $\Re$-part convention (note: $-\mathrm{j} = \mathrm{e}^{-\mathrm{j}\frac{\pi}{2}}$, $\cos(\omega t-\frac{\pi}{2}) = \sin(\omega t)$)

$$p(x,t) = 2 p_A \cos(k_x x) \cos(\omega t)$$
$$v(x,t) = 2 p_A \frac{1}{\rho_0 c}\sin(k_x x) \sin(\omega t)$$

The other boundary condition $v(x=-l) = v_A$ yields the relation between $v_A$ and $p_A$ (note: use complex formulation and $-\sin(-x) = \sin(x)$)

$$v_A = 2 p_A \frac{-\mathrm{j}}{\rho_0 c}\sin(k_x (-l)) = \frac{p_A}{\rho_0 c} 2 \mathrm{j} \sin(k_x l)$$

Below, for simplicity, we used $p_A = 1 \mathrm{N}/\mathrm{m}^2$ and also we normalized the plots, such that the actual imposed energy is not given in absolute values.

In [ ]:
n_samples_per_period = 64
c = 343  # m/s, speed of sound
wave_length = 1  # m
pA = 1  # N / m^2 -> see above how vA is derived from that
rho0 = 1.2041  # kg/m^3

In [ ]:
kx = 2*np.pi / wave_length
w_c = kx
w = w_c * c
Ts = 2*np.pi / (w*n_samples_per_period)  # s, sampling period
x = np.linspace(-2*wave_length, 0, 2**10)

In [ ]:
N = 2**7
x = np.linspace(-2.1334*wave_length, 0, N, endpoint=True)
# boundary condition for rigid wall
p_cpx = 2*pA * np.cos(kx*x)
v_cpx = 2*pA * -1j / (rho0*c) * np.sin(kx*x)

p_re = 2*pA * np.cos(kx*x)
v_re = 2*pA * 1/(rho0*c) * np.sin(kx*x)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), nrows=1, ncols=1)

flag = 'cpx'  # 'cpx', re'

if flag == 'cpx':

    time_phasor_cpx = np.exp(+1j*w*Ts*0)
    ax.plot(x, np.real(p_cpx/pA*time_phasor_cpx), 'C0', label='pressure')
    ax.plot(x, np.real(v_cpx/pA*rho0*c*time_phasor_cpx), 'C1', label='velocity')
    for k in range(n_samples_per_period):
        time_phasor = np.exp(+1j*w*Ts*k)
        ax.plot(x, np.real(p_cpx/pA*time_phasor), 'C0', lw=1)
        ax.plot(x, np.real(v_cpx/pA*rho0*c*time_phasor), 'C1', lw=1)

elif flag == 're':

    time_phasor_cos = np.cos(w*Ts*0)
    time_phasor_sin = np.sin(w*Ts*0)
    ax.plot(x, p_re/pA*time_phasor_cos, 'C0', label='pressure')
    ax.plot(x, v_re/pA*rho0*c*time_phasor_sin, 'C1', label='velocity')
    for k in range(n_samples_per_period):
        time_phasor_cos = np.cos(w*Ts*k)
        time_phasor_sin = np.sin(w*Ts*k)
        ax.plot(x, p_re/pA*time_phasor_cos, 'C0', lw=1)
        ax.plot(x, v_re/pA*rho0*c*time_phasor_sin, 'C1', lw=1)


ax.set_xlabel('x / m')
ax.set_ylabel(r'$\frac{p}{pA}$, $\frac{v}{pA}\cdot\rho_0 c$')
ax.set_xlim(x[0], x[-1])
ax.set_ylim(-2, 0)
ax.set_xticks(np.linspace(-2*wave_length, 0, 5))
ax.set_yticks(np.linspace(-2, 2, 5))
ax.legend()
ax.grid(True)

## 3.2 Animation of standing wave

In [ ]:
flag = 'cpx'  # 'cpx', re'
n_periods = 2
n_samples_per_period = 64
Ts = 2*np.pi / (w*n_samples_per_period)  # s, sampling period


def init():
    ax.set_xlabel('x / m')
    ax.set_ylabel(r'$\frac{p}{pA}$, $\frac{v}{pA}\cdot\rho_0 c$')
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(-2, 0)
    ax.set_xticks(np.linspace(-2*wave_length, 0, 5))
    ax.set_yticks(np.linspace(-2, 2, 5))
    ax.grid(True)


def update(frame):
    ax.clear()
    init()

    if flag == 'cpx':

        time_phasor = np.exp(+1j*w*Ts*frame)
        ax.plot(x, np.real(p_cpx/pA*time_phasor), 'C0', label='pressure')
        ax.plot(x, np.real(v_cpx/pA*rho0*c*time_phasor), 'C1', label='velocity')

    elif flag == 're':

        time_phasor_cos = np.cos(w*Ts*frame)
        time_phasor_sin = np.sin(w*Ts*frame)
        ax.plot(x, p_re/pA*time_phasor_cos, 'C0', label='pressure')
        ax.plot(x, v_re/pA*rho0*c*time_phasor_sin, 'C1', label='velocity')

    ax.set_title('elapsed time {time:4.3f} ms'.format(time=Ts*frame*1000))
    ax.legend(loc='upper left')

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4), nrows=1, ncols=1)
update(0)
init()

ani = FuncAnimation(fig, update,
                    frames=np.arange(n_periods*n_samples_per_period+4),
                    blit=True, interval=50,
                    repeat=False)
plt.show()

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
    rights to this work.
</p>